In [623]:
layer_num = 0

from sklearn.metrics.pairwise import cosine_similarity

num_heads = model.base_model.num_heads
dim = model.base_model.dim
hidden_dim = dim // num_heads

w = model.word_embeddings().detach()
Q = model.base_model.layers[layer_num].Q.weight.detach()
K = model.base_model.layers[layer_num].K.weight.detach()

sims = cosine_similarity(w, Q)
q_tops = sims.argsort(axis=0)[::-1][:5].T

sims = cosine_similarity(w, K)
k_tops = sims.argsort(axis=0)[::-1][:5].T

In [624]:
for h in range(num_heads):
  # print(q_tops[h*(hidden_dim+1):(h+1)*(hidden_dim+1)], k_tops[h*hidden_dim:(h+1)*hidden_dim])
  q_heads = q_tops[h*(hidden_dim+1):(h+1)*(hidden_dim+1)]
  k_heads = k_tops[h*hidden_dim:(h+1)*hidden_dim]

  q_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in q_heads]
  k_tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in k_heads]

  q_lens = [len(''.join(tokens)) for tokens in q_tokens]
  k_lens = [len(''.join(tokens)) for tokens in k_tokens]
  q_max = max(q_lens)
  k_max = max(k_lens)

  for q, k, q_len, k_len in zip(q_tokens, k_tokens, q_lens, k_lens):
    # print(q, k, q_len, k_len)

    print(f"{', '.join(q)}, {' '*(q_max-q_len)}{', '.join(k)}{' '*(k_max-k_len)}")

  print(q_tokens[-1])
  print()

##duction, boosted, cart, emirates, ##●,                       ¦, acheron, [unused500], reapers, ##•                          
flows, ∇, ##esa, ##rh, ##unda,                                 handicapped, complain, ##36, ##oir, 1737                       
senator, ¹, groningen, repertory, winslow,                     flexed, cautiously, ##あ, [unused912], libertadores             
##tower, conor, assent, [unused623], ##isance,                 ##rove, ##resh, ##nco, ##pad, demi                             
manipulated, scarf, [unused106], ##堂, apparel,                 precipitation, schumacher, slashed, 1746, twelfth              
[unused968], sped, た, ##歌, procedural,                         however, elves, ##pressive, likeness, linguist                 
doorstep, blurred, enables, induce, ##！,                       մ, crotch, [unused897], ##₹, kindly                            
：, [unused41], ラ, आ, [unused594],                              [unused512], turnbull, flickering, authorization

In [625]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = 'cointegrated/rubert-tiny'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

print(score(sentence='London is the capital of Great Britain.', model=model, tokenizer=tokenizer)) 
# 4.541251105675365
print(score(sentence='London is the capital of South America.', model=model, tokenizer=tokenizer)) 
# 6.162017238332462

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

4.541282504624356
6.162053232352749


In [121]:
# his = model.initial_states()['histories'][0].expand(data.shape[0], -1, -1)
# data = batch['data'][batch['batch_sizes']]
# lima = model.limas[0]
# theta = model.angle_embedding(data)

# output, next_history = lima(theta, his)
# # output

In [123]:
# batch_size = theta.shape[0]
# total_in = torch.concat([theta[:, None, :].expand(-1, lima.num_heads, -1), his], dim=-1).view(batch_size, -1)

In [124]:
# h = lima.fc_in(total_in)
# h

In [ ]:
import utils.load_corpus as load_corpus
import importlib
load_corpus = importlib.reload(load_corpus)

dataset = load_corpus.load_bookcorpus(tokenizer, cache_dir='~/data1-0756727/cache/huggingface')
# dataset = load_corpus.load_msmarco(tokenizer, cache_dir='~/data1-0756727/cache/huggingface')

dataset['train'][:2]

In [ ]:
batch_size = 32
shuffle = False

class Trainloader:
  def __init__(self, dataset, batch_size):
    self.data = dataset
    self.batch_size = batch_size

  def __len__(self):
    return len(self.data) // batch_size + 1

  def __iter__(self):
    current_start = 0
    for i in range(self.__len__()):
      yield self.data[current_start:current_start+self.batch_size]
      current_start += self.batch_size

# train_dataloader = DataLoader(dataset["train"], batch_size=batch_size, collate_fn=collate)
train_loader = Trainloader((dataset.shuffle() if shuffle else dataset)['train'], batch_size=batch_size)


batch = next(iter(train_loader))
# print(tokenizer.decode(n['data']))
{key: batch[key].shape for key in n}

In [ ]:
his = model.initial_states()
data = batch['data'][batch['batch_sizes']]
theta = model.angle_embedding(data)
output, next_history = model.limas[0](theta, his['histories'][0].expand(data.shape[0], -1, -1))
output

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

proj = PCA().fit_transform(word_repr)
plt.scatter(*proj.T[:2])

norm = np.linalg.norm(word_repr, axis=1)
print(norm.min(), norm.max())
tokenizer.convert_ids_to_tokens(proj.T[:2].argmax(axis=1))

In [ ]:
n = 100

sorted_norm_idx = norm.argsort()
print('mins:', *zip(norm[sorted_norm_idx[:n]], tokenizer.convert_ids_to_tokens(sorted_norm_idx[:n])))
print()
print('maxs:', *zip(norm[sorted_norm_idx[:-n-1:-1]], tokenizer.convert_ids_to_tokens(sorted_norm_idx[:-n-1:-1])))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

proj = PCA().fit_transform(angle_repr)
plt.scatter(*proj.T[:2])

norm = np.linalg.norm(word_repr, axis=1)
print(norm.min(), norm.max())
tokenizer.convert_ids_to_tokens(proj.T[:2].argmax(axis=1))